In [1]:
import cv2
import os
from PIL import Image
from scipy import misc
import tensorflow as tf
import tensorflow.keras as keras
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import gzip
%matplotlib inline
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img


# DATA Preprocessing

In [2]:
#Data Preprocessing

def create_dataset_train(img_folder):
   
    img_data_array=[]
    class_name=[]
    
    for file in os.listdir(img_folder):
        
        image_path = os.path.join(img_folder, file)
        image = load_img(image_path, 'rb')
        image = img_to_array(image)
        
        if image.shape[2] == 3:
            image = image.mean(2)
        img_data_array.append(image)
        name_index = file.split("_")
        name_index = int(name_index[0])
        class_name.append(name_index)
        
    return np.array(img_data_array), np.array(class_name)

def create_dataset_test(img_folder):
   
    img_data_array=[]
    class_name=[]
    
    for file in os.listdir(img_folder):
        
        image_path = os.path.join(img_folder, file)
        image = load_img(image_path, 'rb')
        image = img_to_array(image)
        
        if image.shape[2] == 3:
            image = image.mean(2)
        img_data_array.append(image)
        name_index = file.split(".")
        name_index = int(name_index[0])
        class_name.append(name_index)
        
    return np.array(img_data_array), np.array(class_name)

def normalization(image):
    image = image / image.max()
    return image


training_path = "./02_face_training"
test_path = './02_face_test'

train_data, train_label = create_dataset_train(training_path)
test_data, _ = create_dataset_test(test_path)

train_data = normalization(train_data)
test_data = normalization(test_data)

In [3]:
#preparing test_result_DataFrame

test_index = os.listdir(test_path)
temp = []

for index in test_index:
    index = index.split(".")
    temp.append(int(index[0]))
    
test_index = temp

In [4]:
print('Shape of Train images :',train_data.shape)
print('Shape of Train labels : ', train_label.shape)
print('Shape of Test images : ', test_data.shape)

Shape of Train images : (1050, 56, 46)
Shape of Train labels :  (1050,)
Shape of Test images :  (700, 56, 46)


# Model

In [5]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(56, 46)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(351, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 2576)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               659712    
_________________________________________________________________
dense_1 (Dense)              (None, 351)               90207     
Total params: 749,919
Trainable params: 749,919
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(train_data, train_label, epochs = 40)

Epoch 1/40
33/33 [==============================] - 1s 7ms/step - loss: 5.9728 - accuracy: 9.5238e-04
Epoch 2/40
33/33 [==============================] - 0s 6ms/step - loss: 5.8030 - accuracy: 0.0038
Epoch 3/40
33/33 [==============================] - 0s 6ms/step - loss: 5.6706 - accuracy: 0.0067
Epoch 4/40
33/33 [==============================] - 0s 6ms/step - loss: 5.4511 - accuracy: 0.0057
Epoch 5/40
33/33 [==============================] - 0s 6ms/step - loss: 5.1892 - accuracy: 0.0200
Epoch 6/40
33/33 [==============================] - 0s 6ms/step - loss: 4.9046 - accuracy: 0.0371
Epoch 7/40
33/33 [==============================] - 0s 8ms/step - loss: 4.5669 - accuracy: 0.0629
Epoch 8/40
33/33 [==============================] - 0s 6ms/step - loss: 4.2288 - accuracy: 0.0876
Epoch 9/40
33/33 [==============================] - 0s 6ms/step - loss: 3.9258 - accuracy: 0.1105
Epoch 10/40
33/33 [==============================] - 0s 6ms/step - loss: 3.6517 - accuracy: 0.1514
Epoch 11/40
33/

# Training Validation

In [8]:
predictions = model.predict(train_data)

In [9]:
print(predictions.shape)

(1050, 351)


In [10]:
predict_label = []
for image in predictions:
    predict_label.append(np.argmax(image))

In [11]:
predict_label = np.array(predict_label)

In [12]:
def accuracy(original, prediction):
    accuracy = original == prediction
    accuracy = np.count_nonzero(accuracy)
    
    return accuracy / original.shape[0]
    
accuracy(train_label, predict_label)

0.9019047619047619

# Test

In [13]:
predictions = model.predict(test_data)

In [14]:
print(predictions.shape)

(700, 351)


In [15]:
test_label = []
for image in predictions:
    test_label.append(np.argmax(image))

In [16]:
test_label = np.array(test_label)

In [17]:
#결과
print(test_label)

[ 41 153 268 231 245  51 326 212  51 273 267 299 211  34  55 227 251 289
 324  34 212 194 280 266 231  25 201 319 136 271 211 220 278 158 216 287
 180 225 263 206 119 210 310 337 123 177 344 168 241 347 239 211 164 186
 109 226 162  34 128 222  61 192 285 220 231 154 202 128 211 344 111 310
 152 195 220 308 209 100  56 299 308 268 268 144 290 224 201 123 284 225
 118 270 254 240 137 276 236 101 200 103 151 347 169 345  93 116 225 347
 328 167 203 261 220 217 288 294 278 218 196 226 226 328 231 211 268 281
 276 201 210 301 215 238 151 188 151 179 248 231 236 220  11 278 212 195
 234 178 152 310  73 231 234  65 212  92 118 242 226 333  33 215 328  84
 274 264 220 328 152 192 117 202 183 162 183 211   3 329 227 316 317 236
 283 339 300  65 226 109 218 211 154 216 223  76 225 229 262 243 215 190
 209 263 234 313 231 328 285 324 305 119  85 258 104  20 167 100  34 220
 271 215 348 338 119 250 180 250 345 252 280 161 295 119 263 220  56 219
 175 248 184 225 284 151 330 341 210 308 234 315 17

# Extract Pandas DataFrame

In [18]:
test_result_df = pd.DataFrame([x for x in zip(test_index, test_label)], columns=['Image', 'Answer'])

In [19]:
test_result_df.head()

,Image,Answer
0,1,41
1,10,153
2,100,268
3,101,231
4,102,245


In [20]:
test_result_df = test_result_df.sort_values(by="Image")
test_result_df.head()

,Image,Answer
0,1,41
111,2,261
222,3,180
333,4,238
444,5,329


In [21]:
test_result_df.to_csv("./result.csv", index=False)

PermissionError: [Errno 13] Permission denied: './result.csv'